# **Regresión Logística para Imagenes**

Los modelos de Machine Learning son modelos matemáticos que tratan de resolver un problema de regresión.

In [19]:
# --- Inicialización y Estructura de Datos ---

# Instalar ipycanvas (si no está instalado).
# !pip install ipycanvas ipywidgets numpy pandas scikit-learn

# Importar librerías
import numpy as np
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display
from ipycanvas import Canvas
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns

# 1. Estructura de datos global
X_data = [] # Imágenes aplanadas (100 pixeles)
y_labels = [] # Etiquetas (dígitos)

# 2. Configuración del Canvas
canvas_size = 280 # Tamaño visual del canvas
target_size = 10 # Resolución de los datos (target_size x target_size). Resolución en la que se codificaran las imagenes para el modelo a entrenar
brush_size = 20 # Grosor del lápiz

print(f"Librerías importadas. Tamaño de los datos objetivo: {target_size}x{target_size}.")

Librerías importadas. Tamaño de los datos objetivo: 10x10.


## **Captura de Datos**

La generación de datos es esencial para poder lograr el entrenamiento de modelos de Machine Learning.
En esta ocasión es necesaria la generación de los mismos de una manera interactiva para que tengas la oportunidad de ver la importancia que esta etapa representa.
En la parte de abajo aparecerá un cuadro blanco en el que podrás escribir/dibujar un digito usando el cursor de la computadora. A cada uno de los digitos escritos se les debe de asignar una etiqueta con el nombre de la clase o digito que se intentó escribir.

In [20]:
# --- Widget Interactivo de Dibujo ---

# 1. Crear el Canvas
canvas = Canvas(width=canvas_size, height=canvas_size, sync_image_data=True)
canvas.layout.border = '2px solid black'
canvas.fill_style = 'white'
canvas.fill_rect(0, 0, canvas_size, canvas_size)
canvas.line_width = brush_size
canvas.stroke_style = 'black'
canvas.line_cap = 'round'

# 2. Configuración del ratón para dibujar
is_drawing = False

def handle_mouse_down(x, y):
    global is_drawing
    is_drawing = True
    canvas.begin_path()
    canvas.move_to(x, y)

def handle_mouse_move(x, y):
    if is_drawing:
        canvas.line_to(x, y)
        canvas.stroke()

def handle_mouse_up(x, y):
    global is_drawing
    is_drawing = False

canvas.on_mouse_down(handle_mouse_down)
canvas.on_mouse_move(handle_mouse_move)
canvas.on_mouse_up(handle_mouse_up)

# 3. Widgets de Control
# CORRECCIÓN: Ajustamos width y style para visibilidad
label_input = widgets.IntText(
    value=0, 
    description='Dígito:', 
    min=0, max=9, 
    layout=widgets.Layout(width='150px') # Hacemos la caja más ancha
)
status_label = widgets.Label(value="Esperando dibujo...")
clear_button = widgets.Button(description="Limpiar")
save_button = widgets.Button(description="Guardar Muestra")

# 4. Lógica de Guardado (CORRECCIÓN FINAL)
def save_sample(b):
    global_vars = globals()
    X_data = global_vars['X_data']
    y_labels = global_vars['y_labels']
    target_size = global_vars['target_size']
    canvas = global_vars['canvas']
    
    label = global_vars['label_input'].value # Accedemos al valor del widget
    
    try:
        # --- NUEVA LÓGICA DE CAPTURA Y PROCESAMIENTO ---
        
        # 1. Obtener los datos como un arreglo NumPy de píxeles (RGBA)
        # Esto nos da un arreglo (280, 280, 4)
        data_np = canvas.get_image_data() 
        
        if data_np is None:
            global_vars['status_label'].value = "ERROR: El canvas no contiene datos. Dibuja algo primero."
            return

        # 2. Convertir el arreglo NumPy a una imagen PIL
        from PIL import Image
        # data_np ya es un ndarray, pero lo convertimos a PIL para usar sus herramientas de redimensión
        img = Image.fromarray(np.uint8(data_np), 'RGBA') 
        
        # 3. Procesamiento de la imagen
        # Convertir a escala de grises y redimensionar
        img = img.convert('L').resize((target_size, target_size)) 
        img_array = np.array(img, dtype=np.float32)
        
        # 4. Invertir colores y normalizar
        # Invertimos porque el fondo es blanco (alto valor) y queremos el dígito (negro) con alto valor
        img_array = 255 - img_array
        img_array = img_array / 255.0
        
        # Aplanar y guardar
        X_data.append(img_array.flatten())
        y_labels.append(label)
        
        # Actualizar estado y limpiar
        global_vars['status_label'].value = f"Muestra #{len(X_data)} (Dígito: {label}) guardada. ¡Dibuje otra!"
        global_vars['clear_canvas'](None)
        
    except Exception as e:
        # Mensaje de error detallado si falla el procesamiento
        global_vars['status_label'].value = f"ERROR en Python: {type(e).__name__}: {e}"
        print(f"Error completo en la función save_sample: {e}")

# 5. Lógica de Limpiar
def clear_canvas(b):
    canvas.clear()
    canvas.fill_style = 'white'
    canvas.fill_rect(0, 0, canvas_size, canvas_size) # Restaurar fondo blanco
    
# 6. Conectar Eventos (VERIFICACIÓN CLAVE)
# Esta parte es la que conecta el click del botón con la función Python
clear_button.on_click(clear_canvas) 
save_button.on_click(save_sample) # <-- Esta línea debe ejecutarse sin errores

# 7. Mostrar Widgets
controls = widgets.HBox([label_input, save_button, clear_button])
display(status_label, controls, canvas)

Label(value='Esperando dibujo...')

Canvas(height=280, layout=Layout(border_bottom='2px solid black', border_left='2px solid black', border_right=…

## **¿Que se capturó?**

Generalmente es útil tener una visualización general de las imágenes que se capturaron. Esto nos puede dar claridad de los datos que tenemos y podemos dimensionar la complejidad del problema.

In [21]:
# --- Visualización de Muestras Aleatorias ---

import numpy as np
import matplotlib.pyplot as plt
import random

def mostrar_muestras_aleatorias(X_data, y_labels, num_muestras=10):
    """
    Muestra una cantidad especificada de imágenes (10x10) seleccionadas al azar
    del dataset recolectado.
    """
    
    total_muestras = len(X_data)
    
    if total_muestras == 0:
        print("El dataset aún está vacío. ¡Por favor, dibuje y guarde algunas muestras primero!")
        return

    # Limitar el número de muestras a mostrar si es mayor que el total disponible
    if num_muestras > total_muestras:
        num_muestras = total_muestras
        print(f"Advertencia: Solo hay {total_muestras} muestras disponibles. Mostrando todas.")

    # 1. Seleccionar índices aleatorios
    indices_aleatorios = random.sample(range(total_muestras), num_muestras)

    # 2. Configurar el tamaño del array (cuadrícula) para la visualización
    # Calculamos el número de filas y columnas para una disposición cuadrada
    num_cols = int(np.ceil(np.sqrt(num_muestras)))
    num_rows = int(np.ceil(num_muestras / num_cols))
    
    plt.figure(figsize=(num_cols * 1.5, num_rows * 1.8))
    plt.suptitle(f"Muestras Aleatorias de Datos (Total: {total_muestras})", fontsize=14)

    # 3. Iterar sobre los índices seleccionados y mostrar las imágenes
    for i, idx in enumerate(indices_aleatorios):
        # Seleccionar el vector de píxeles
        vector_pixeles = X_data[idx]
        etiqueta = y_labels[idx]
        
        # Reformar el vector (1x100) a la matriz de imagen (10x10)
        imagen_10x10 = vector_pixeles.reshape(10, 10) # <-- Clave de la visualización
        
        # Crear un subplot
        plt.subplot(num_rows, num_cols, i + 1)
        
        # Mostrar la imagen en escala de grises (cmap='gray')
        plt.imshow(imagen_10x10, cmap='gray', interpolation='nearest')
        
        # Añadir la etiqueta
        plt.title(f"Etiqueta: {etiqueta}", fontsize=10)
        
        # Ocultar los ejes para una mejor visualización de la cuadrícula
        plt.axis('off')

    plt.tight_layout(rect=[0, 0, 1, 0.96]) # Ajustar diseño para evitar superposición con el título
    plt.show()

# --- Ejemplo de uso (Ejecuta esta parte de la celda) ---

# Muestra 15 imágenes aleatorias. Puedes cambiar este número.
muestras_a_mostrar = 10 

# Asegúrate de que X_data y y_labels son accesibles desde esta celda.
mostrar_muestras_aleatorias(X_data, y_labels, num_muestras=muestras_a_mostrar)

El dataset aún está vacío. ¡Por favor, dibuje y guarde algunas muestras primero!


## **Si se captura el dibujo, ¿que contiene la imagen?**

Una imagen es una representación gráfica de datos que para nosotros como humanos es fácil de compreder (en la mayoría de las veces).

Pero para una computadora, directamente las imagenes no se pueden procesar, deben de interpretarse de una manera numérica. Entonces un digito en este ejemplo es una matriz de numeros que representan el digito y que se nos presenta como una imagen.

In [14]:
indx = 0
img_res = X_data[indx].reshape(target_size, target_size)
fig = plt.figure(figsize=(8,6))
sns.heatmap(img_res,cmap=plt.cm.gray_r,annot=img_res)
#plt.imshow(digits.images[digito],cmap=plt.cm.gray_r)
plt.title('Digits: %i' % (y_labels[indx]))
plt.show()

IndexError: list index out of range


# Regresión Logística

La **Regresión Logística** es un modelo estadístico utilizado para problemas de clasificación binaria. A diferencia de la regresión lineal, que predice valores continuos, la regresión logística estima la **probabilidad** de que una observación pertenezca a una clase específica (por ejemplo, 0 o 1).

Este modelo se basa en la función logística (sigmoide), que transforma cualquier valor real en un rango entre 0 y 1, ideal para representar probabilidades.

## Fórmula del Modelo

La probabilidad de que la variable dependiente ( $y$ ) sea igual a 1, dado un conjunto de características ( $x$ ), se expresa como:

$$
P(y = 1 \mid x) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_n x_n)}}
$$

Donde:
- $( \beta_0 )$ es el intercepto.
-  $(\beta_i)$ son los coeficientes asociados a cada característica $( x_i )$.
- La función sigmoide garantiza que $( 0 \leq P(y=1|x) \leq 1 )$.

## Interpretación

Cada coeficiente $( \beta_i )$ indica la influencia de la característica $( x_i )$ en la probabilidad de pertenecer a la clase positiva. Un valor positivo aumenta la probabilidad, mientras que uno negativo la disminuye. Este enfoque es ampliamente utilizado en problemas como clasificación de imágenes, diagnóstico médico y análisis de riesgo.


In [ ]:
# --- Entrenamiento y Evaluación del Modelo ---

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def entrenar_y_evaluar_modelo():
    """Entrena un modelo de Regresión Logística con los datos recolectados."""
    
    global X_data, y_labels # Acceder a los datos recolectados en Celda 2
    
    # 1. Verificar la cantidad de datos
    if len(X_data) < 10:
        print(f"¡Necesitas más datos! Solo tienes {len(X_data)} muestras. Se recomienda al menos 30 para un resultado significativo.")
        return

    # 2. Dividir los datos
    X = np.array(X_data)
    y = np.array(y_labels)

    # Separar en conjuntos de entrenamiento (80%) y prueba (20%)
    # stratify=y asegura que la proporción de cada dígito sea similar en ambos conjuntos
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print("--- Preparación de Datos ---")
    print(f"Total de muestras: {len(X)}")
    print(f"Muestras de Entrenamiento: {len(X_train)}")
    print(f"Muestras de Prueba: {len(X_test)}")
    
    # 3. Entrenar el Modelo (Regresión Logística)
    print("\n--- Iniciando Entrenamiento de Regresión Logística ---")
    # Aumentamos max_iter para asegurar la convergencia en datos de alta dimensionalidad
    modelo_entrenado = LogisticRegression(max_iter=500, solver='lbfgs')
    modelo_entrenado.fit(X_train, y_train)

    # 4. Evaluación
    y_pred = modelo_entrenado.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print("\n--- Resultados del Modelo ---")
    print(f"Precisión (Accuracy) en el conjunto de prueba: {accuracy:.2f}")

    print("\n¡ANÁLISIS DE DATOS! Estos resultados demuestran la importancia de:")
    print("1. **Tamaño de la muestra:** Más datos = mejor modelo.")
    print("2. **Extracción de Características:** Las imágenes de 10x10 píxeles hacen que este problema sea difícil para un modelo simple.")
    
    return modelo_entrenado # Opcional: devolver el modelo para usarlo en nuevas predicciones

# Ejecutar el entrenamiento
modelo_entrenado = entrenar_y_evaluar_modelo()

In [ ]:
# --- Widget de Predicción Interactiva ---

import numpy as np
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display
from ipycanvas import Canvas
import matplotlib.pyplot as plt


if 'modelo_entrenado' not in globals() or modelo_entrenado is None:
    print("¡Advertencia! El modelo no ha sido entrenado. Por favor, ejecuta la Celda 3 primero.")


# --- Configuración del Canvas

# 1. Crear el Canvas para Predicción
prediction_canvas = Canvas(width=canvas_size, height=canvas_size, sync_image_data=True)
prediction_canvas.layout.border = '2px solid black'
prediction_canvas.fill_style = 'white'
prediction_canvas.fill_rect(0, 0, canvas_size, canvas_size)
prediction_canvas.line_width = brush_size
prediction_canvas.stroke_style = 'black'
prediction_canvas.line_cap = 'round'

# 2. Configuración del ratón para dibujar (igual que antes)
is_drawing_pred = False

def handle_mouse_down_pred(x, y):
    global is_drawing_pred
    is_drawing_pred = True
    prediction_canvas.begin_path()
    prediction_canvas.move_to(x, y)

def handle_mouse_move_pred(x, y):
    if is_drawing_pred:
        prediction_canvas.line_to(x, y)
        prediction_canvas.stroke()

def handle_mouse_up_pred(x, y):
    global is_drawing_pred
    is_drawing_pred = False

prediction_canvas.on_mouse_down(handle_mouse_down_pred)
prediction_canvas.on_mouse_move(handle_mouse_move_pred)
prediction_canvas.on_mouse_up(handle_mouse_up_pred)

# 3. Widgets de Control para Predicción
predict_button = widgets.Button(description="Predecir Dígito")
clear_pred_button = widgets.Button(description="Limpiar")
prediction_output_label = widgets.Label(value="Dibuje un dígito y presione Predecir.")

# Contenedor para mostrar las probabilidades
prob_output = widgets.Output() 

# 4. Lógica de Predicción
def predict_digit(b):
    if 'modelo_entrenado' not in globals() or modelo_entrenado is None:
        prediction_output_label.value = "ERROR: El modelo no está entrenado. ¡Entrénalo en la Celda 3!"
        return

    try:
        # Obtener los datos del dibujo (misma lógica de preprocesamiento que en save_sample)
        data_np = prediction_canvas.get_image_data()
        
        if data_np is None:
            prediction_output_label.value = "ERROR: No hay dibujo para predecir."
            return

        img = Image.fromarray(np.uint8(data_np), 'RGBA')
        img = img.convert('L').resize((target_size, target_size))
        img_array = np.array(img, dtype=np.float32)
        
        img_array = 255 - img_array # Invertir colores
        img_array = img_array / 255.0 # Normalizar
        
        # Aplanar la imagen para la predicción del modelo
        input_feature = img_array.flatten().reshape(1, -1) # Reshape para que sea (1, 100)
        
        # Realizar la predicción
        predicted_digit = modelo_entrenado.predict(input_feature)[0]
        
        # Obtener las probabilidades de cada clase
        probabilities = modelo_entrenado.predict_proba(input_feature)[0]
        class_labels = modelo_entrenado.classes_ # Obtener las etiquetas de clase (0-9)
        
        prediction_output_label.value = f"Predicción: {predicted_digit}"

        # Mostrar las probabilidades en un gráfico
        with prob_output:
            prob_output.clear_output(wait=True)
            plt.figure(figsize=(6, 3))
            plt.bar(class_labels, probabilities)
            plt.title('Probabilidad de cada Dígito')
            plt.xlabel('Dígito')
            plt.ylabel('Probabilidad')
            plt.xticks(class_labels) # Asegura que se muestren todos los dígitos en el eje X
            plt.grid(axis='y', alpha=0.75)
            plt.show()

    except Exception as e:
        prediction_output_label.value = f"ERROR en predicción: {type(e).__name__}: {e}"

# 5. Lógica de Limpiar para el Canvas de Predicción
def clear_prediction_canvas(b):
    prediction_canvas.clear()
    prediction_canvas.fill_style = 'white'
    prediction_canvas.fill_rect(0, 0, canvas_size, canvas_size)
    prediction_output_label.value = "Dibuje un dígito y presione Predecir."
    with prob_output:
        prob_output.clear_output() # Limpiar el gráfico de probabilidades

# 6. Conectar Eventos
predict_button.on_click(predict_digit)
clear_pred_button.on_click(clear_prediction_canvas)

# 7. Mostrar Widgets
prediction_controls = widgets.HBox([predict_button, clear_pred_button])
display(prediction_output_label, prediction_controls, prediction_canvas, prob_output)

## **¿Qué ha aprendido el Modelo? (Visualización de Parámetros)**

La Regresión Logística asigna un peso a cada píxel de la imagen ($10 \times 10 = 100$ píxeles de entrada) para determinar si pertenece a una clase (dígito) o no.

Si un peso ($\beta$) es positivo y grande, ese píxel es una fuerte evidencia de que el dígito es la clase objetivo.  

Si el peso es negativo y grande, ese píxel es una fuerte evidencia de que el dígito no es la clase objetivo.  

Podemos tomar los 100 pesos ($\beta$s) asociados a un dígito específico (por ejemplo, el dígito '1') y reorganizarlos en una matriz de $10 \times 10$ píxeles. Esto nos mostrará la "plantilla" o "prototipo" que el modelo ha aprendido para esa clase.


In [ ]:
# --- Celda 6: Visualización de los Parámetros del Modelo ---

import matplotlib.pyplot as plt
import numpy as np

def visualizar_parametros_modelo(modelo):
    """
    Visualiza los coeficientes (pesos) de la Regresión Logística como imágenes
    para mostrar qué patrón ha aprendido el modelo para cada dígito.
    """
    
    if 'modelo_entrenado' not in globals() or modelo is None:
        print("ERROR: El modelo no está entrenado. Por favor, ejecuta la Celda 3 primero.")
        return

    # Los coeficientes están almacenados en la propiedad coef_
    # Tienen la forma (número de clases, número de características) -> (10, 100)
    coeficientes = modelo.coef_
    clases = modelo.classes_
    num_clases = len(clases)
    
    # Configurar la cuadrícula para las 10 imágenes (2 filas x 5 columnas)
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    plt.suptitle("Prototipos (Pesos) Aprendidos por la Regresión Logística", fontsize=16)

    # Iterar sobre cada clase (dígito)
    for i, ax in enumerate(axes.flat):
        if i >= num_clases:
            ax.axis('off')
            continue
            
        # 1. Obtener los 100 coeficientes para la clase actual
        pesos_clase = coeficientes[i]
        
        # 2. Reformar el vector (1x100) a la matriz de imagen (10x10)
        prototipo = pesos_clase.reshape(10, 10)
        
        # 3. Visualizar la matriz de pesos
        # Usamos cmap='seismic' o 'RdBu' para mostrar valores positivos (rojo) y negativos (azul).
        # Un valor central (cerca de cero) indica un píxel irrelevante.
        im = ax.imshow(prototipo, cmap='seismic', 
                       interpolation='nearest', 
                       # Aseguramos que el centro de color (blanco/cero) esté en 0
                       vmin=prototipo.min(), vmax=prototipo.max()) 
        
        # 4. Añadir título y barra de color
        ax.set_title(f"Clase: {clases[i]}", fontsize=12)
        ax.axis('off')

    # Añadir una barra de color única para todo el gráfico
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7]) # Posición [izquierda, abajo, ancho, alto]
    fig.colorbar(im, cax=cbar_ax, label='Peso del Píxel (Prototipo)')
    
    fig.subplots_adjust(right=0.88, wspace=0.1, hspace=0.3)
    plt.show()

# Ejecutar la visualización (asumiendo que modelo_entrenado es la variable global)
if 'modelo_entrenado' in globals():
    visualizar_parametros_modelo(modelo_entrenado)
else:
    print("Por favor, entrena el modelo en la Celda 3 antes de visualizar los parámetros.")